### SOUMAHORO Youssouf
### Enseignant: Olaf KOUAMO

## TP: scoring-lowcost

    objectif: redire l'appentence des clients a des transport lowcoast.
              Pour cela, nous utiliserons la librairie Ml de spark

In [2]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f

### Desciption des fichiers de données

### Travail à faire

1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [3]:
# Lecture des fichiers
perimetre = spark_session.read.csv("data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("data_clients/sample_visites.csv", header=True)

In [4]:
histo_train_raw.dtypes

[('ID_CLIENT', 'string'),
 ('nb_od', 'string'),
 ('mean_nb_passagers', 'string'),
 ('mean_duree_voyage', 'string'),
 ('mean_mt_voyage', 'string'),
 ('mean_tarif_loisir', 'string'),
 ('mean_classe_1', 'string'),
 ('mean_pointe', 'string'),
 ('mean_depart_we', 'string')]

In [36]:
histo_client_raw.dtypes

[('ID_CLIENT', 'string'),
 ('anciennete', 'string'),
 ('recence_cmd', 'string'),
 ('AGE', 'string'),
 ('LBL_STATUT_CLT', 'string'),
 ('LBL_GEO_AIR', 'string'),
 ('LBL_GRP_SEGMENT_NL', 'string'),
 ('LBL_SEG_COMPORTEMENTAL', 'string'),
 ('LBL_GEO_TRAIN', 'string'),
 ('LBL_SEGMENT_ANTICIPATION', 'string'),
 ('FLG_CMD_CARTE_1225', 'string')]

In [5]:
## Fonction pour transformer pour caster 
def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast(cast_type))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

Nous allons transformer certaines variables de notre ensemble de données en **double** ou **int**

In [7]:
# cast_to_float
client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225','anciennete','AGE']

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["recence_cmd"],
                                  client_cols_to_keep,
                                 cast_type='double')
#cast_to_int
client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225','recence_cmd']

histo_client = cast_columns_of_df(histo_client,
                                  ["anciennete",'AGE'],
                                  client_cols_to_keep,
                                 cast_type='int')

In [42]:
histo_client.dtypes

[('ID_CLIENT', 'string'),
 ('LBL_STATUT_CLT', 'string'),
 ('LBL_GEO_AIR', 'string'),
 ('LBL_SEG_COMPORTEMENTAL', 'string'),
 ('LBL_GEO_TRAIN', 'string'),
 ('LBL_GRP_SEGMENT_NL', 'string'),
 ('LBL_SEGMENT_ANTICIPATION', 'string'),
 ('FLG_CMD_CARTE_1225', 'string'),
 ('recence_cmd', 'double'),
 ('anciennete', 'int'),
 ('AGE', 'int')]

In [44]:
visites.dtypes

[('ID_CLIENT', 'string'),
 ('days_since_last_visit', 'double'),
 ('tx_conversion', 'double')]

In [45]:
histo_train.dtypes

[('ID_CLIENT', 'string'),
 ('nb_od', 'double'),
 ('mean_nb_passagers', 'double'),
 ('mean_duree_voyage', 'double'),
 ('mean_mt_voyage', 'double'),
 ('mean_tarif_loisir', 'double'),
 ('mean_classe_1', 'double'),
 ('mean_pointe', 'double'),
 ('mean_depart_we', 'double')]

### faire une jointure entre les informations des différentes tables.
     NB: on conservera tous les clients de la table perimetre.
        En effet, ce sont les cleints qu'on souhaite scorer

In [8]:
df = perimetre.join(histo_client,on='ID_CLIENT',how='left_outer')\
       .join(visites,on='ID_CLIENT',how='left_outer')\
       .join(histo_train,on='ID_CLIENT',how='left_outer')\
        .join(histo_lowcost,on='ID_CLIENT',how='left_outer')
len(df.columns)

24

In [96]:
# Vérification
perimetre.count()==df.count()

True

#### combien a t'on de features quatitatives, qualitatives 
     Pour nous faciliter la tache, nousa allons créer une fonction qui va nous renvoyer la liste des
     features continues et celle des features qualitatives

In [9]:
# Cette fonction prend en paramètre df.dtypes et renvoi les listes de features continues et qualitative

def type_feature(df_dtype):
    continous_columns=list()
    qualifies_columns=list()
    
    for col in range(len(df_dtype)):
        
        if(df_dtype[col][1]=='string'):
            qualifies_columns.append(df_dtype[col][0])
            
        else:
            continous_columns.append(df_dtype[col][0])
    
    return(continous_columns,qualifies_columns)


In [10]:
features=type_feature(df.dtypes)
features[0] # les features continues
#features[1]

['recence_cmd',
 'anciennete',
 'AGE',
 'days_since_last_visit',
 'tx_conversion',
 'nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'flg_cmd_lowcost',
 'flg_track_nl_lowcost',
 'flg_track_nl']

In [100]:
print("features continues: {}".format(len(type_feature(df.dtypes)[0])))
print("features qualitatives: {}".format(len(type_feature(df.dtypes)[1])))

features continues: 16
features qualitatives: 8


#### Les features avec valeurs manquantes
     # Cette fonction renvoie un data frame contenant les features le nombre de valeurs manquantes
     qu'elles contiennent

In [19]:
def find_feature_missing_values(df):
    from pyspark.sql.functions import col,sum
    return df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas().iloc[:,0:10]

find_feature_missing_values(df)

,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,recence_cmd,anciennete
0,0,78998,162977,155160,163010,79522,157822,10283,1484,55


### Remplacement des valeurs manquantes
    Nous allons remplacer les valeurs manquantes dans les features par le -1 si la feature est 
    qualitative ou par la moyenne si elle est continue

In [11]:
# Appel à la fonction
features=type_feature(df.dtypes)
features[1].append('flg_cmd_lowcost')
features[0].pop(13)

'flg_cmd_lowcost'

In [12]:
# dico des moyennes des features continues
dic_mean={}
for feat in features[0]:
    dic_mean[feat]= df.groupBy().avg(feat).take(1)[0][0]


df1=df.select([f.when(df[feature].isNotNull(),df[feature]).otherwise("-1").alias(feature) \
               for feature in features[1]]+[f.when(df[feature].isNotNull(),\
                                df[feature]).otherwise(dic_mean[feature]).alias(feature)\
                               for feature in features[0]])

#### Autres fonctions pour remplacer les valeurs manquantes

In [ ]:
# Fonction to replace missing values
def replace_missing_values1(df,qualifies_columns, continuous_columns):
    dict_mean = {feat: df.select(f.mean(feat)).collect()[0][0] 
                 for feat in continuous_columns}
    return df.select([f.when(df[feature].isNotNull(), df[feature])\
                      .otherwise('-1').alias(feature) for feature in qualifies_columns]\
                     +[f.when(df[feature].isNotNull(), df[feature])\
                       .otherwise(dict_mean[feature]).alias(feature) 
                       for feature in continuous_columns])

# Another fonction
def replace_missing_values2(df,qualifies_columns, continuous_columns):
    
    return df.select([f.when(df[feature].isNotNull(), df[feature])\
                      .otherwise('-1').alias(feature) for feature in qualifies_columns]\
                     +[f.when(df[feature].isNotNull(), df[feature])\
                       .otherwise(df.select(f.mean(df[feature])).collect()[0][0])\
                       .alias(feature) for feature in continuous_columns])

In [55]:
df1.printSchema()

root
 |-- ID_CLIENT: string (nullable = true)
 |-- LBL_STATUT_CLT: string (nullable = true)
 |-- LBL_GEO_AIR: string (nullable = true)
 |-- LBL_SEG_COMPORTEMENTAL: string (nullable = true)
 |-- LBL_GEO_TRAIN: string (nullable = true)
 |-- LBL_GRP_SEGMENT_NL: string (nullable = true)
 |-- LBL_SEGMENT_ANTICIPATION: string (nullable = true)
 |-- FLG_CMD_CARTE_1225: string (nullable = true)
 |-- flg_cmd_lowcost: string (nullable = true)
 |-- recence_cmd: double (nullable = true)
 |-- anciennete: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- days_since_last_visit: double (nullable = true)
 |-- tx_conversion: double (nullable = true)
 |-- nb_od: double (nullable = true)
 |-- mean_nb_passagers: double (nullable = true)
 |-- mean_duree_voyage: double (nullable = true)
 |-- mean_mt_voyage: double (nullable = true)
 |-- mean_tarif_loisir: double (nullable = true)
 |-- mean_classe_1: double (nullable = true)
 |-- mean_pointe: double (nullable = true)
 |-- mean_depart_we: double

 Les differentes modalites de la feature **LBL_STATUT_CLT**

In [13]:
df1.select(df["LBL_STATUT_CLT"]).distinct().show()

+--------------------+
|      LBL_STATUT_CLT|
+--------------------+
|         Moyen moins|
|Non present dans ...|
|    Nouveau prospect|
|            Prospect|
|          Tres petit|
|                null|
|               Petit|
|             Inactif|
|       Nouveau actif|
|               Grand|
|          Tres grand|
|          Moyen plus|
+--------------------+



### Reconstitution des données
     Nous allons maintenant reconstituer notre jeu de données en créant des features pour répondre à 
     notre problématique

In [103]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds
df = input_df(df1)

Quelles sont les differentes valeurs de notre label : flg_cmd_lowcost

In [104]:
df.select("flg_cmd_lowcost").distinct().show()

+---------------+
|flg_cmd_lowcost|
+---------------+
|              0|
|              1|
+---------------+



In [58]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

##### features engineering et modélisation

Pour contruire un modèle en Spark il transforme le dataframe en un bon format pour que spark comprenne

In [105]:
# StringIndexer permet d'encoder les variables quali: la modalité la présente est encodée par 0 ensuite la 2eme
#plus présente par 1 ainsi de suite
def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(df)


### Prélevons un sample de data pour notre modelisation

In [106]:
train = data.sample(False,0.01,seed = 0)
test = data.sample(False,0.005,seed = 0)
print("train size: ",train.count())
print("test size: ",test.count())

train size:  10954
test size:  5458


### Logistic regression

quelle est le label est renseigne pour la modelisation?

In [107]:
#labelCol = la variable cible
#featureCol = les variables prédictives
lr = LogisticRegression(labelCol="flg_cmd_lowcostIndex", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)

Ajustement de modele de regression logistique et calcul les coefficients de notre modele.

In [108]:
model_lr = lr.fit(train)
# predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage
predictions = model_lr.transform(test)

In [109]:
predictions.select(['flg_cmd_lowcostIndex','prediction']).show()

+--------------------+----------+
|flg_cmd_lowcostIndex|prediction|
+--------------------+----------+
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 1.0|       1.0|
|                 1.0|       1.0|
|                 0.0|       0.0|
|                 1.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 1.0|       0.0|
|                 0.0|       0.0|
+--------------------+----------+
only showing top 20 rows



### Evaluation du modèle
    Evaluer soit meme le score en calculant le nombre de VP, FP, VN et FN
    on calculera alors le score qui VP+VN/VP+VN+FP+FN
    nb: la prediction est automatiquement creee dans le data set et correspond à la colonne prediction

In [65]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='flg_cmd_lowcostIndex', metricName='areaUnderROC')

In [110]:
from pyspark.mllib.evaluation import MulticlassMetrics
import pandas as pd

results = predictions.select(['prediction', 'flg_cmd_lowcostIndex'])
labels_and_predictions = results.rdd
metrics = MulticlassMetrics(labels_and_predictions)

# AUC 
AUC = evaluator.evaluate(predictions)
print("Model AUC: %.3f%%" % (AUC * 100))
# matrice de confusion
cm = metrics.confusionMatrix().toArray()
# Accuracy
acc = (cm[0][0]+cm[1][1])/(cm.sum())
print("Model accuracy: %.3f%%" % (acc * 100))
#recall
rec = cm[0][0]/(cm[0][0]+cm[0][1])
print("Model recall: %.3f%%" % (rec* 100))
#precision
prec = cm[0][0]/(cm[0][0]+cm[1][0])
print("Model precision: %.3f%%" % (prec * 100))

Model AUC: 89.227%
Model accuracy: 85.562%
Model recall: 99.980%
Model precision: 96.146%


#### Ecrivons maintenant une fonction pour faire nos les moèles
     Cette fonction prend en entrée le classifier que l'on veut ajuster, le jeu de données et le 
     label et renvoie un data frame contenant les métriques d'évaluation 
     (AUC, precision, recall, accuracy)

In [1]:
def compute_model(df,clf,label):
    
    """compute score in a classification modelisation.
       clf: classifier
       df: data set
       label: target
    """
    
    train = df.sample(False,0.01,42)
    test = df.sample(False,0.005,42)

    model = clf.fit(train)
    predictions = model.transform(test)
    
    results = predictions.select(['prediction', label])
    labels_and_predictions = results.rdd
    metrics = MulticlassMetrics(labels_and_predictions)
    
    AUC = evaluator.evaluate(predictions)
    cm = metrics.confusionMatrix().toArray()
    acc = round((cm[0][0]+cm[1][1])/(cm.sum()),2)
    rec = round(cm[0][0]/(cm[0][0]+cm[0][1]),2)
    prec = round(cm[0][0]/(cm[0][0]+cm[1][0]),2)
    
    #res = {"AUC":AUC, "accuracy":acc, "precision": prec, "recall":rec}
    res = pd.DataFrame([AUC, acc,  rec, prec],index=["AUC","accuracy","precision","recall"],
                       columns=["values"])
    
    return res

### Application de la fonction compute_model

#### Logistic Regression

In [112]:
clf_lr = LogisticRegression(labelCol="flg_cmd_lowcostIndex", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)

compute_model(data,clf_lr,'flg_cmd_lowcostIndex')

,values
AUC,0.919728
accuracy,0.830000
precision,1.000000
recall,0.960000


##### Random Forest

In [114]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)
compute_model(data,classifier,'flg_cmd_lowcostIndex')

,values
AUC,1.00
accuracy,0.81
precision,1.00
recall,0.99


### Conclusion
    A l'issue de ce TP nous avons appris à manipuler des données avec l'outil SPARK et à construire des
    modèles de prédictions avec le package ml de SPARK. 